In [2]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("C:\SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [5]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
rf = RandomForestClassifier(n_estimators=50,max_depth=20,n_jobs=-1)

In [8]:
rf_model = rf.fit(x_train,y_train)

In [13]:
sorted(zip(rf_model.feature_importances_, x_train.columns), reverse = True)

[(0.0768122168185551, 'body_len'),
 (0.034389375391025524, 3134),
 (0.03328403744181578, 1803),
 (0.024891613736703223, 7350),
 (0.021814230688581362, 7027),
 (0.021097990334927513, 6285),
 (0.020975256578183058, 2031),
 (0.020317910662531528, 2171),
 (0.018365645750384855, 392),
 (0.01658310874261337, 5078),
 (0.01455116023758722, 3443),
 (0.012368668308428112, 295),
 (0.011120729285325748, 4796),
 (0.01097225750518485, 5988),
 (0.010431326441727458, 375),
 (0.010156302066062484, 6050),
 (0.009879815223163915, 1361),
 (0.009532845497757868, 5453),
 (0.009469045805725775, 7218),
 (0.009329980220647814, 5005),
 (0.008958151082519237, 690),
 (0.008776997183358488, 7782),
 (0.008757572918984541, 1881),
 (0.008751940457689338, 5724),
 (0.008153386240862212, 2095),
 (0.00805338209026332, 2299),
 (0.007532591936423932, 294),
 (0.007120812828620584, 2060),
 (0.007057208097424063, 6971),
 (0.007011658521632709, 397),
 (0.007008693927815514, 6746),
 (0.006939338962206555, 436),
 (0.006932958799

In [15]:
pred = rf_model.predict(x_test)
pred

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype=object)

In [16]:
p,r,f,s = score(y_test,pred,pos_label='spam',average='binary')

In [17]:
print(p)
print(r)
print(f)
print(s)

1.0
0.5357142857142857
0.6976744186046512
None


In [19]:
((sum(pred == y_test))/len(pred))*100

94.16517055655295

In [24]:
def train_rf(n,d):
    rf = RandomForestClassifier(n_estimators=n,max_depth=d,n_jobs=-1)
    rf_model = rf.fit(x_train,y_train)
    pred = rf_model.predict(x_test)
    p,r,f,s = score(y_test,pred,pos_label='spam',average='binary')
    print('n_estimators :' + str(n))
    print('max_deapth :' + str(d))
    print('precision :' + str(p))
    print('recall :' + str(r))
    print('fscore :' + str(f))
    print('accuracy: ' + str(((sum(pred == y_test))/len(pred))*100)) 

In [25]:
count = 1
for nest in [10,50,100]:
    for dep in [10,20,30,None]:
        print(count)
        count+=1
        train_rf(nest,dep)
        print('\n')

1
n_estimators :10
max_deapth :10
precision :1.0
recall :0.2714285714285714
fscore :0.42696629213483145
accuracy: 90.84380610412927


2
n_estimators :10
max_deapth :20
precision :1.0
recall :0.5857142857142857
fscore :0.7387387387387387
accuracy: 94.7935368043088


3
n_estimators :10
max_deapth :30
precision :1.0
recall :0.6714285714285714
fscore :0.8034188034188035
accuracy: 95.87073608617594


4
n_estimators :10
max_deapth :None
precision :0.9805825242718447
recall :0.7214285714285714
fscore :0.8312757201646089
accuracy: 96.31956912028726


5
n_estimators :50
max_deapth :10
precision :1.0
recall :0.2714285714285714
fscore :0.42696629213483145
accuracy: 90.84380610412927


6
n_estimators :50
max_deapth :20
precision :1.0
recall :0.5428571428571428
fscore :0.7037037037037037
accuracy: 94.25493716337523


7
n_estimators :50
max_deapth :30
precision :1.0
recall :0.6571428571428571
fscore :0.7931034482758621
accuracy: 95.69120287253142


8
n_estimators :50
max_deapth :None
precision :1.0


In [26]:
from sklearn.model_selection import GridSearchCV

In [28]:
param = {'n_estimators' : [10,150,300],
         'max_depth' : [30,60,90,None] }

In [29]:
rf2 = RandomForestClassifier()

In [33]:
gs = GridSearchCV(rf2,param,cv=5,n_jobs=-1)

In [34]:
gs_fit = gs.fit(X_features,data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending=False)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\utils\deprecatio

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
7,25.436888,0.657456,0.782688,0.715866,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.979372,0.978437,0.977538,...,0.975211,0.004168,1,0.999326,0.999102,0.999326,0.999326,0.998653,0.999147,0.000262
11,43.492394,4.795329,0.422990,0.074587,None,300,"{'max_depth': None, 'n_estimators': 300}",0.978475,0.974843,0.975741,...,0.973954,0.003373,2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
3,2.862816,0.193975,0.191159,0.019066,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.978475,0.968553,0.977538,...,0.973774,0.003880,3,0.991015,0.991917,0.991693,0.994163,0.994836,0.992725,0.001494
8,48.293183,2.879754,0.518221,0.032507,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977578,0.979335,0.973046,...,0.973415,0.004610,4,0.999551,0.999326,0.999102,0.999775,0.999326,0.999416,0.000229
10,26.035264,1.122326,0.319224,0.028825,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977578,0.973944,0.973944,...,0.972876,0.003094,5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
4,20.984390,0.381379,0.345316,0.049059,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.973991,0.973944,0.973046,...,0.972157,0.003169,6,0.993935,0.992366,0.995734,0.993264,0.993040,0.993668,0.001148
5,40.334658,0.693513,0.610973,0.168443,60,300,"{'max_depth': 60, 'n_estimators': 300}",0.976682,0.970350,0.971249,...,0.971080,0.003443,7,0.993935,0.992591,0.995285,0.994612,0.992591,0.993803,0.001078
6,3.751350,0.258993,0.249291,0.036045,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.971300,0.978437,0.970350,...,0.970181,0.004803,8,0.997754,0.997755,0.998877,0.997306,0.998877,0.998114,0.000645
9,4.630757,1.647397,0.225680,0.062490,None,10,"{'max_depth': None, 'n_estimators': 10}",0.967713,0.977538,0.972147,...,0.970181,0.005057,8,0.997978,0.997755,0.996857,0.998204,0.996408,0.997440,0.000690
2,23.998585,0.363771,0.344124,0.008645,30,300,"{'max_depth': 30, 'n_estimators': 300}",0.970404,0.959569,0.964960,...,0.960841,0.006128,10,0.975292,0.973956,0.973731,0.975752,0.975528,0.974852,0.000839
